In [ ]:
import torch
import torchaudio
import numpy as np
import os
import pandas as pd
import json

In [ ]:
from tqdm import tqdm

In [ ]:
def MFCC(waveform, sample_rate):
    mfcc = torchaudio.compliance.kaldi.mfcc(waveform=waveform, num_ceps= 12,  
                                     cepstral_lifter=22, channel=0, 
                                     dither = 0.0, energy_floor=0.0,
                                     frame_length=20, frame_shift=10, 
                                     high_freq=sample_rate/2,low_freq = 40, num_mel_bins=25, 
                                     sample_frequency=sample_rate, use_energy=True, vtln_low =60, vtln_high = 7200, 
                                     window_type="hamming")
    return mfcc

In [35]:
set_loc = "./set_new_data/"
data_loc = "./"
ann_loc = "./breathe_annotations/"
for i in os.listdir(set_loc):
    set_full = np.loadtxt(set_loc + i, dtype=str)
    for j in tqdm(set_full):
        A = {}
        Breath_1 = []
        Breath_2 = []
        Breath_3 = []
        waveform, sample_rate = torchaudio.load(data_loc+j+".wav")
        try: 
            txt_file = pd.read_csv(ann_loc+j+".txt", sep="\t", header = None)
            labels = txt_file.iloc[:,2]
            start = txt_file.iloc[:,0]
            stop = txt_file.iloc[:,1]
            for k,l in enumerate(labels):
                try:
                    if l[:3] == "Bre":
                        length = stop[k]-start[k]                                             
                        
                        Breath_1.append(MFCC(waveform[:,int(round(start[k]*sample_rate)):int(round((stop[k]+(length)/4)*sample_rate))], sample_rate).numpy().tolist())
                        Breath_2.append(MFCC(waveform[:,int(round((start[k]+(length/4))*sample_rate)):int(round((stop[k]+(3*length)/4)*sample_rate))], sample_rate).numpy().tolist())
                        Breath_3.append(MFCC(waveform[:,int(round((start[k]+(3*length)/4)*sample_rate)):int(round((stop[k])*sample_rate))], sample_rate).numpy().tolist())

                        
                except:
                    print(j)
                
            A['Breath_1'] = Breath_1
            A['Breath_2'] = Breath_2
            A['Breath_3'] = Breath_3

            with open("./breath_mfcc/"+j+".json", "w") as outfile:  
                json.dump(A, outfile) 

        except:
            pass




100%|██████████| 285/285 [22:51<00:00,  4.81s/it]
